<a href="https://colab.research.google.com/github/apchavezr/prueba_trabajo/blob/main/Sistema_Congestion_Docker_K8s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema de predicción de congestión vehicular con Docker y Kubernetes

Este cuaderno desarrolla un sistema básico de predicción de congestión vehicular usando un modelo de clasificación en Python.
Se encapsula con Docker y se despliega en Kubernetes para escalar automáticamente según la demanda.

El flujo es el siguiente:
1. Entrenar modelo con datos simulados.
2. Crear API REST con Flask.
3. Contenerizar con Docker.
4. Desplegar en Kubernetes con autoescalado.

## Paso 1: Simulación de datos y entrenamiento del modelo

In [1]:
# Simulación de datos de tráfico
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import joblib

np.random.seed(42)
n = 500

# Variables: hora del día, día de la semana, clima, eventos, congestión (0: baja, 1: alta)
df = pd.DataFrame({
    'hora': np.random.randint(0, 24, size=n),
    'dia_semana': np.random.randint(0, 7, size=n),
    'lluvia': np.random.randint(0, 2, size=n),
    'evento_masivo': np.random.randint(0, 2, size=n)
})

# Simular variable objetivo
df['congestion'] = ((df['hora'].between(7,9)) | (df['hora'].between(17,19))) & (df['lluvia'] | df['evento_masivo'])
df['congestion'] = df['congestion'].astype(int)

# Entrenar modelo
X = df.drop(columns='congestion')
y = df['congestion']
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X, y)

# Guardar modelo
joblib.dump(modelo, 'modelo_congestion.pkl')
print("Modelo entrenado y guardado.")

Modelo entrenado y guardado.


## Paso 2: API Flask

In [ ]:
# app.py
from flask import Flask, request, jsonify
import pandas as pd
import joblib

app = Flask(__name__)
modelo = joblib.load('modelo_congestion.pkl')

@app.route('/predecir', methods=['POST'])
def predecir():
    datos = request.get_json()
    df = pd.DataFrame(datos)
    pred = modelo.predict(df)
    return jsonify(pred.tolist())

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

## Paso 3: Dependencias (requirements.txt)

In [ ]:
flask
scikit-learn
pandas
joblib

## Paso 4: Dockerfile

In [ ]:
# Dockerfile
FROM python:3.10
WORKDIR /app
COPY requirements.txt .
RUN pip install -r requirements.txt
COPY . .
RUN python train_model.py
EXPOSE 5000
CMD ["python", "app.py"]

## Paso 5: Despliegue en Kubernetes (YAML básico con autoescalado)

In [ ]:
# deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: congestion-app
spec:
  replicas: 2
  selector:
    matchLabels:
      app: congestion
  template:
    metadata:
      labels:
        app: congestion
    spec:
      containers:
      - name: congestion
        image: tu_usuario/congestion:latest
        ports:
        - containerPort: 5000
---
apiVersion: v1
kind: Service
metadata:
  name: congestion-service
spec:
  selector:
    app: congestion
  ports:
    - protocol: TCP
      port: 80
      targetPort: 5000
  type: LoadBalancer
---
apiVersion: autoscaling/v2
kind: HorizontalPodAutoscaler
metadata:
  name: congestion-hpa
spec:
  scaleTargetRef:
    apiVersion: apps/v1
    kind: Deployment
    name: congestion-app
  minReplicas: 2
  maxReplicas: 10
  metrics:
  - type: Resource
    resource:
      name: cpu
      target:
        type: Utilization
        averageUtilization: 50

## ✅ Conclusión

Este sistema de predicción vehicular demuestra cómo combinar Python, Docker y Kubernetes para escalar automáticamente en función de la demanda. Puede adaptarse fácilmente a un entorno real conectándose a sensores de tráfico y bases de datos urbanas.